<a href="https://colab.research.google.com/github/GaohuiXu/llm-cookbook/blob/main/content/%E5%BF%85%E4%BF%AE%E4%B8%80-Prompt%20Engineering%20For%20Developers/5.%20%E6%8E%A8%E6%96%AD%20Inferring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第五章 推断

在这节课中，你将从产品评论和新闻文章中推断情感和主题。



<div class="toc">
 <ul class="toc-item">
     <li><span><a href="#一引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
     <li>
         <span><a href="#二情感推断与信息提取" data-toc-modified-id="二、情感推断与信息提取">二、情感推断与信息提取</a></span>
         <ul class="toc-item">
             <li><span><a href="#21-情感倾向分析" data-toc-modified-id="2.1 情感倾向分析">2.1 情感倾向分析</a></span></li>
             <li><span><a href="#22-识别情感类型" data-toc-modified-id="2.2 识别情感类型">2.2 识别情感类型</a></span></li>
             <li><span><a href="#23-识别愤怒" data-toc-modified-id="2.3 识别愤怒">2.3 识别愤怒</a></span></li>
             <li><span><a href="#24-商品信息提取" data-toc-modified-id="2.4 商品信息提取">2.4 商品信息提取</a></span></li>
             <li><span><a href="#25-综合完成任务" data-toc-modified-id="2.5 综合完成任务">2.5 综合完成任务</a></span></li>
             </ul>
         </li>
     <li><span><a href="#三主题推断" data-toc-modified-id="三、主题推断">三、主题推断</a></span></li>
     <ul class="toc-item">
             <li><span><a href="#31-推断讨论主题" data-toc-modified-id="3.1 推断讨论主题">3.1 推断讨论主题</a></span></li>
             <li><span><a href="#32-为特定主题制作新闻提醒" data-toc-modified-id="3.2 为特定主题制作新闻提醒">3.2 为特定主题制作新闻提醒</a></span></li>
             </ul>
     </ul>
</div>

## 一、引言

推断任务可以看作是模型接收文本作为输入，并执行某种分析的过程。其中涉及提取标签、提取实体、理解文本情感等等。如果你想要从一段文本中提取正面或负面情感，在传统的机器学习工作流程中，需要收集标签数据集、训练模型、确定如何在云端部署模型并进行推断。这样做可能效果还不错，但是执行全流程需要很多工作。而且对于每个任务，如情感分析、提取实体等等，都需要训练和部署单独的模型。

LLM 的一个非常好的特点是，对于许多这样的任务，你只需要编写一个 Prompt 即可开始产出结果，而不需要进行大量的工作。这极大地加快了应用程序开发的速度。你还可以只使用一个模型和一个 API 来执行许多不同的任务，而不需要弄清楚如何训练和部署许多不同的模型。

In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.75.0
    Uninstalling openai-1.75.0:
      Successfully uninstalled openai-1.75.0


In [2]:
import openai
# 导入第三方库

openai.api_key  = "sk-de071700bc094713b4567d255658606f"
openai.api_base = "https://api.deepseek.com/v1"
# 设置 API_KEY, 请替换成您自己的 API_KEY

In [3]:
def get_completion(prompt, model="deepseek-reasoner"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## 二、情感推断与信息提取
### 2.1 情感分类

以电商平台关于一盏台灯的评论为例，可以对其传达的情感进行二分类（正向/负向）。

In [ ]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [4]:
# 中文
lamp_review_zh = """
我需要一盏漂亮的卧室灯，这款灯具有额外的储物功能，价格也不算太高。\
我很快就收到了它。在运输过程中，我们的灯绳断了，但是公司很乐意寄送了一个新的。\
几天后就收到了。这款灯很容易组装。我发现少了一个零件，于是联系了他们的客服，他们很快就给我寄来了缺失的零件！\
在我看来，Lumina 是一家非常关心顾客和产品的优秀公司！
"""

现在让我们来编写一个 Prompt 来分类这个评论的情感。如果我想让系统告诉我这个评论的情感是什么，只需要编写 “以下产品评论的情感是什么” 这个 Prompt ，加上通常的分隔符和评论文本等等。

然后让我们运行一下。结果显示这个产品评论的情感是积极的，这似乎是非常正确的。虽然这盏台灯不完美，但这个客户似乎非常满意。这似乎是一家关心客户和产品的伟大公司，可以认为积极的情感似乎是正确的答案。

In [ ]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


In [5]:
# 中文
prompt = f"""
以下用三个反引号分隔的产品评论的情感是什么？

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

这个产品评论的情感总体上是积极的。尽管消费者在运输过程中遇到了问题（灯绳断裂）和产品组装时缺少零件，但重点在于公司（Lumina）的快速响应和优质服务。评论中多次使用正面表述，例如：
- "价格也不算太高"（对价格的认可）
- "公司很乐意寄送了一个新的"（服务态度好）
- "很快就给我寄来了缺失的零件"（高效解决问题）
- "非常关心顾客和产品的优秀公司"（直接赞扬）

虽然存在产品运输和质检问题，但消费者最终通过积极的客服体验形成了整体正面评价，属于带有建设性批评的积极情感（positive with constructive criticism）。


如果你想要给出更简洁的答案，以便更容易进行后处理，可以在上述 Prompt 基础上添加另一个指令：*用一个单词回答：「正面」或「负面」*。这样就只会打印出 “正面” 这个单词，这使得输出更加统一，方便后续处理。

In [ ]:
prompt = f"""
What is the sentiment of the following product review,
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

positive


In [6]:
prompt = f"""
以下用三个反引号分隔的产品评论的情感是什么？

用一个单词回答：「正面」或「负面」。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

正面


### 2.2 识别情感类型

仍然使用台灯评论，我们尝试另一个 Prompt 。这次我需要模型识别出评论作者所表达的情感，并归纳为列表，不超过五项。

In [ ]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

satisfied, grateful, impressed, content, pleased


In [8]:
# 中文
prompt = f"""
识别以下评论的作者表达的情感。包含不超过五种情感类型。将答案格式化为以逗号分隔的单词列表。

回答请使用中文

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

满意,赞赏,信任,认可,愉快


大型语言模型非常擅长从一段文本中提取特定的东西。在上面的例子中，评论所表达的情感有助于了解客户如何看待特定的产品。

### 2.3 识别愤怒

对于很多企业来说，了解某个顾客是否非常生气很重要。所以产生了下述分类问题：以下评论的作者是否表达了愤怒情绪？因为如果有人真的很生气，那么可能值得额外关注，让客户支持或客户成功团队联系客户以了解情况，并为客户解决问题。

In [ ]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

No


In [9]:
# 中文
prompt = f"""
以下评论的作者是否表达了愤怒？评论用三个反引号分隔。给出是或否的答案。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

否


上面这个例子中，客户并没有生气。注意，如果使用常规的监督学习，如果想要建立所有这些分类器，不可能在几分钟内就做到这一点。我们鼓励大家尝试更改一些这样的 Prompt ，也许询问客户是否表达了喜悦，或者询问是否有任何遗漏的部分，并看看是否可以让 Prompt 对这个灯具评论做出不同的推论。

### 2.4 商品信息提取

接下来，让我们从客户评论中提取更丰富的信息。信息提取是自然语言处理（NLP）的一部分，与从文本中提取你想要知道的某些事物相关。因此，在这个 Prompt 中，我要求它识别以下内容：购买物品和制造物品的公司名称。

同样，如果你试图总结在线购物电子商务网站的许多评论，对于这些评论来说，弄清楚是什么物品、谁制造了该物品，弄清楚积极和消极的情感，有助于追踪特定物品或制造商收获的用户情感趋势。

在下面这个示例中，我们要求它将响应格式化为一个 JSON 对象，其中物品和品牌作为键。

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


In [10]:
# 中文
prompt = f"""
从评论文本中识别以下项目：
- 评论者购买的物品
- 制造该物品的公司

评论文本用三个反引号分隔。将你的响应格式化为以 “物品” 和 “品牌” 为键的 JSON 对象。
如果信息不存在，请使用 “未知” 作为值。
让你的回应尽可能简短。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

```json
{"物品": "卧室灯", "品牌": "Lumina"}
```


如上所示，它会说这个物品是一个卧室灯，品牌是 Luminar，你可以轻松地将其加载到 Python 字典中，然后对此输出进行其他处理。

### 2.5 综合完成任务

提取上述所有信息使用了 3 或 4 个 Prompt ，但实际上可以编写单个 Prompt 来同时提取所有这些信息。

In [ ]:
prompt = f"""
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


In [11]:
# 中文
prompt = f"""
从评论文本中识别以下项目：
- 情绪（正面或负面）
- 审稿人是否表达了愤怒？（是或否）
- 评论者购买的物品
- 制造该物品的公司

评论用三个反引号分隔。将您的响应格式化为 JSON 对象，以 “Sentiment”、“Anger”、“Item” 和 “Brand” 作为键。
如果信息不存在，请使用 “未知” 作为值。
让你的回应尽可能简短。
将 Anger 值格式化为布尔值。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

```json
{"Sentiment":"正面","Anger":false,"Item":"卧室灯","Brand":"Lumina"}
```


这个例子中，我们告诉它将愤怒值格式化为布尔值，然后输出一个 JSON。您可以自己尝试不同的变化，或者甚至尝试完全不同的评论，看看是否仍然可以准确地提取这些内容。

## 三、主题推断

大型语言模型的另一个很酷的应用是推断主题。给定一段长文本，这段文本是关于什么的？有什么话题？以以下一段虚构的报纸报道为例。

In [ ]:
story = """
In a recent survey conducted by the government,
public sector employees were asked to rate their level
of satisfaction with the department they work at.
The results revealed that NASA was the most popular
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings,
stating, "I'm not surprised that NASA came out on top.
It's a great place to work with amazing people and
incredible opportunities. I'm proud to be a part of
such an innovative organization."

The results were also welcomed by NASA's management team,
with Director Tom Johnson stating, "We are thrilled to
hear that our employees are satisfied with their work at NASA.
We have a talented and dedicated team who work tirelessly
to achieve our goals, and it's fantastic to see that their
hard work is paying off."

The survey also revealed that the
Social Security Administration had the lowest satisfaction
rating, with only 45% of employees indicating they were
satisfied with their job. The government has pledged to
address the concerns raised by employees in the survey and
work towards improving job satisfaction across all departments.
"""

In [12]:
# 中文
story_zh = """
在政府最近进行的一项调查中，要求公共部门的员工对他们所在部门的满意度进行评分。
调查结果显示，NASA 是最受欢迎的部门，满意度为 95％。

一位 NASA 员工 John Smith 对这一发现发表了评论，他表示：
“我对 NASA 排名第一并不感到惊讶。这是一个与了不起的人们和令人难以置信的机会共事的好地方。我为成为这样一个创新组织的一员感到自豪。”

NASA 的管理团队也对这一结果表示欢迎，主管 Tom Johnson 表示：
“我们很高兴听到我们的员工对 NASA 的工作感到满意。
我们拥有一支才华横溢、忠诚敬业的团队，他们为实现我们的目标不懈努力，看到他们的辛勤工作得到回报是太棒了。”

调查还显示，社会保障管理局的满意度最低，只有 45％的员工表示他们对工作满意。
政府承诺解决调查中员工提出的问题，并努力提高所有部门的工作满意度。
"""

### 3.1 推断讨论主题

上面是一篇虚构的关于政府工作人员对他们工作机构感受的报纸文章。我们可以让它确定五个正在讨论的主题，用一两个字描述每个主题，并将输出格式化为逗号分隔的列表。

In [ ]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long.

Format your response as a list of items separated by commas.

Text sample: ```{story}```
"""
response = get_completion(prompt)
print(response)

government survey, public sector employees, job satisfaction, NASA, Social Security Administration


In [ ]:
response.split(sep=',')

['government survey',
 ' public sector employees',
 ' job satisfaction',
 ' NASA',
 ' Social Security Administration']

In [13]:
# 中文
prompt = f"""
确定以下给定文本中讨论的五个主题。

每个主题用1-2个单词概括。

输出时用逗号分割每个主题。

给定文本: ```{story_zh}```
"""
response = get_completion(prompt)
print(response)

员工满意度调查,NASA 满意度,员工反馈,管理层回应,社会保障管理局


### 3.2 为特定主题制作新闻提醒

假设我们有一个新闻网站或类似的东西，这是我们感兴趣的主题：NASA、地方政府、工程、员工满意度、联邦政府等。假设我们想弄清楚，针对一篇新闻文章，其中涵盖了哪些主题。可以使用这样的prompt：确定以下主题列表中的每个项目是否是以下文本中的主题。以 0 或 1 的形式给出答案列表。

In [ ]:
topic_list = [
    "nasa", "local government", "engineering",
    "employee satisfaction", "federal government"
]

In [ ]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: ```{story}```
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [ ]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


In [15]:
# 中文
prompt = f"""
判断主题列表中的每一项是否是给定文本中的一个话题，

以列表的形式给出答案，每个主题用 0 或 1。

主题列表：美国航空航天局、当地政府、工程、员工满意度、联邦政府

输出形式如引号所示 '''主题：0/1 '''

给定文本: ```{story_zh}```
"""
response = get_completion(prompt)
print(response)

'''  
美国航空航天局:1  
当地政府:0  
工程:0  
员工满意度:1  
联邦政府:1  
'''


有结果可见，这个故事是与关于 NASA 、员工满意度、联邦政府有关，而与当地政府的、工程学无关。这在机器学习中有时被称为 Zero-Shot （零样本）学习算法，因为我们没有给它任何标记的训练数据。仅凭 Prompt ，它就能确定哪些主题在新闻文章中有所涵盖。

如果我们想生成一个新闻提醒，也可以使用这个处理新闻的过程。假设我非常喜欢 NASA 所做的工作，就可以构建一个这样的系统，每当 NASA 新闻出现时，输出提醒。

In [16]:
topic_dict = {i.split('：')[0]: int(i.split('：')[1]) for i in response.split(sep='\n')}
if topic_dict['美国航空航天局'] == 1:
    print("提醒: 关于美国航空航天局的新消息")

IndexError: list index out of range

这就是关于推断的全部内容了，仅用几分钟时间，我们就可以构建多个用于对文本进行推理的系统，而以前则需要熟练的机器学习开发人员数天甚至数周的时间。这非常令人兴奋，无论是对于熟练的机器学习开发人员，还是对于新手来说，都可以使用 Prompt 来非常快速地构建和开始相当复杂的自然语言处理任务。